In [ ]:
import os
import pickle
import warnings
import numpy as np
import pandas as pd
import altair as alt
import seaborn as sns

In [ ]:
urls = [
    '/content/drive/MyDrive/36104/at3_data/1 to 15 May 2013.csv',
    '/content/drive/MyDrive/36104/at3_data/1 May to 15 May 2014.csv',
    '/content/drive/MyDrive/36104/at3_data/1 May to 15 May 2015.csv',
    '/content/drive/MyDrive/36104/at3_data/1 to 15 Feb 2013.csv',
    '/content/drive/MyDrive/36104/at3_data/1 to 15 Feb 2014.csv',
    '/content/drive/MyDrive/36104/at3_data/1 to 15 Feb 2015.csv',
    '/content/drive/MyDrive/36104/at3_data/15 to 30 Nov 2013.csv',
    '/content/drive/MyDrive/36104/at3_data/15 to 30 Nov 2014.csv',
    '/content/drive/MyDrive/36104/at3_data/15 to 30 Nov 2015.csv'
]

In [ ]:
dfs = []
for url in urls:
    df_temp = pd.read_csv(url)
    dfs.append(df_temp)

df = pd.concat(dfs, axis=0, ignore_index=True)

<ipython-input-3-4f59f44643f0>:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(url)
<ipython-input-3-4f59f44643f0>:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(url)
<ipython-input-3-4f59f44643f0>:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(url)


In [ ]:
post_code_df = pd.read_csv('/content/drive/MyDrive/36104/post_code.csv')
unique_post_code_df = post_code_df.groupby('Place').agg(
    {'Code': 'min'}
).reset_index()
unique_post_code_df.rename(columns={'Code': 'post_code'}, inplace=True)
unique_post_code_df['Place'] = unique_post_code_df['Place'].str.strip()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446729 entries, 0 to 446728
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Date & Time  446729 non-null  object 
 1   RFID         446729 non-null  int64  
 2   Serial       446222 non-null  float64
 3   Bin Type     446729 non-null  object 
 4   Schedule     446729 non-null  object 
 5   Weight       293143 non-null  float64
 6   Truck        446729 non-null  object 
 7   Zone         446286 non-null  float64
 8   Reference    441421 non-null  object 
 9   Billing      0 non-null       float64
 10  Unit         144469 non-null  object 
 11  House        446681 non-null  object 
 12  Street       446729 non-null  object 
 13  Suburb       446662 non-null  object 
dtypes: float64(4), int64(1), object(9)
memory usage: 47.7+ MB


In [ ]:
df_cleaned = df.copy()
# drop missing values
df_cleaned = df_cleaned.dropna(subset=['Weight'])
df_cleaned = df_cleaned[df_cleaned['Weight']<=75].reset_index(drop=True)

# recycle waste is the waste in Bin starts with R
recycled_mask = df_cleaned['Bin Type'].str.startswith('R')
df_cleaned.loc[recycled_mask, 'Recycled Weight'] = df_cleaned['Weight']
df_cleaned['Recycled Weight'].fillna(0, inplace=True)   

# convert Date & Time column to datetime format
df_cleaned['date_time'] = pd.to_datetime(df_cleaned['Date & Time'], format="%d/%m/%Y %H:%M")
df_cleaned['date'] = pd.to_datetime(df_cleaned['date_time']).dt.date
df_cleaned['week'] = df_cleaned['date_time'].dt.isocalendar().week
df_cleaned['year'] = df_cleaned['date_time'].dt.year
df_cleaned['dayofweek'] = df_cleaned['date_time'].dt.day_name()
df_cleaned = df_cleaned.sort_values(by='date').reset_index(drop=True)

In [ ]:
string_columns = [
 'Bin Type',
 'Schedule',
 'Truck',
 'Reference',
 'Unit',
 'House',
 'Street',
 'Suburb'
]
# strip trailing/ leading white spaces
for col in string_columns:
    df_cleaned[col] = df_cleaned[col].str.strip()

In [ ]:
df_cleaned = df_cleaned.merge(
    unique_post_code_df[['Place', 'post_code']],
    left_on='Suburb',
    right_on='Place'
)

In [ ]:
df_cleaned.head(5)

,Date & Time,RFID,Serial,Bin Type,Schedule,Weight,Truck,Zone,Reference,Billing,...,Street,Suburb,Recycled Weight,date_time,date,week,year,dayofweek,Place,post_code
0,1/02/2013 10:14,159731100,106289.0,G-0140,Every Friday,5.6,AT17QA,19.0,1889,NaN,...,Forrest Street,Chifley,0.0,2013-02-01 10:14:00,2013-02-01,5,2013,Friday,Chifley,2036
1,1/02/2013 10:14,159745806,105596.0,G-0140,Every Friday,13.5,AT17QA,19.0,1887,NaN,...,Forrest Street,Chifley,0.0,2013-02-01 10:14:00,2013-02-01,5,2013,Friday,Chifley,2036
2,1/02/2013 10:14,161117492,247814.0,R-0240,Fortnightly B Friday,5.4,AT44WE,19.0,2160,NaN,...,Macquarie Street,Chifley,5.4,2013-02-01 10:14:00,2013-02-01,5,2013,Friday,Chifley,2036
3,1/02/2013 10:13,159731071,105595.0,G-0140,Every Friday,3.0,AT17QA,19.0,1885,NaN,...,Forrest Street,Chifley,0.0,2013-02-01 10:13:00,2013-02-01,5,2013,Friday,Chifley,2036
4,1/02/2013 10:13,161129034,246744.0,R-0240,Fortnightly B Friday,61.2,AT44WE,19.0,2158,NaN,...,Macquarie Street,Chifley,61.2,2013-02-01 10:13:00,2013-02-01,5,2013,Friday,Chifley,2036


In [ ]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205212 entries, 0 to 205211
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Date & Time      205212 non-null  object        
 1   RFID             205212 non-null  int64         
 2   Serial           205047 non-null  float64       
 3   Bin Type         205212 non-null  object        
 4   Schedule         205212 non-null  object        
 5   Weight           205212 non-null  float64       
 6   Truck            205212 non-null  object        
 7   Zone             204987 non-null  float64       
 8   Reference        172221 non-null  object        
 9   Billing          0 non-null       float64       
 10  Unit             70976 non-null   object        
 11  House            205175 non-null  object        
 12  Street           205212 non-null  object        
 13  Suburb           205212 non-null  object        
 14  Recycled Weight  205

In [ ]:
df_cleaned.columns

Index(['Date & Time', 'RFID', 'Serial', 'Bin Type', 'Schedule', 'Weight',
       'Truck', 'Zone', 'Reference', 'Billing', 'Unit', 'House', 'Street',
       'Suburb', 'Recycled Weight', 'date_time', 'date', 'week', 'year',
       'dayofweek', 'Place', 'post_code'],
      dtype='object')

In [ ]:
selected_columns = [
    'date_time', 'date', 'week', 'year',
    'RFID', 'Serial', 'Bin Type', 'Schedule', 'Weight',
    'Truck', 'Zone', 'Reference', 'Billing', 'House', 'Street',
    'Suburb', 'post_code', 'Recycled Weight', 
    'dayofweek', 
]
df_cleaned = df_cleaned[selected_columns]

In [ ]:
df_cleaned['address'] = df_cleaned['House'] + '-' + df_cleaned['Street'] + '-' + df_cleaned['Suburb']
df_cleaned['address'] = df_cleaned['address'].str.lower()

property_detail_df = pd.read_csv('/content/drive/MyDrive/36104/at3_data/property_detail.csv')
property_detail_df['End House Alpha'] = property_detail_df['End House Alpha'].str.strip().fillna('')
property_detail_df['address'] = property_detail_df['Start House Number'].astype(str) + property_detail_df['End House Alpha'] + '-' + property_detail_df['Street'] + '-' + property_detail_df['Suburb']
property_detail_df['address'] = property_detail_df['address'].str.lower()
property_detail_unique_df = property_detail_df[['Property Type', 'address', 'Total Units']].groupby('address').agg(
    {'Property Type': 'max',
     'Total Units': 'max'}
).reset_index()

# property_detail_unique_df.to_csv('/content/drive/MyDrive/36104/Assignment 3 - Shared/property_detail_unique.csv')

df_cleaned_house_details = df_cleaned.merge(property_detail_unique_df,
                                            on='address', how='left')

# calculate number of house number
df_cleaned_house_details['house_len'] = df_cleaned_house_details['House'].str.strip('-').str.count('-') + 1

df_cleaned_house_details.loc[df_cleaned_house_details['Total Units'].isna(), 'Total Units'] = df_cleaned_house_details['house_len']
# some house has missing house number (37 records) => force total unit to be 1
df_cleaned_house_details['Total Units'].fillna(1, inplace=True)
# total unit can not be 0 => force to be 1
df_cleaned_house_details.loc[df_cleaned_house_details['Total Units']==0, 'Total Units'] = 1


In [ ]:
df_cleaned_house_details.to_csv('/content/drive/MyDrive/36104/Assignment 3 - Shared/assignment_3_cleaned_data_with_property_details.csv')

In [ ]:
df_cleaned.to_csv('/content/drive/MyDrive/36104/Assignment 3 - Shared/assignment_3_cleaned_data.csv')